In [ ]:
from openlocationcode import openlocationcode as olc
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="carwash_locator", timeout=10)

def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language="en", addressdetails=True)
        if location and location.address:
            return location.address
    except Exception as e:
        print(f"Reverse geocoding failed for ({lat}, {lon}): {e}")
    return None

def get_nearby_named_elements(lat, lon, radius=50):
    query = f"""
    [out:json][timeout:25];
    (
      node(around:{radius}, {lat}, {lon})["name"];
      way(around:{radius}, {lat}, {lon})["name"];
      relation(around:{radius}, {lat}, {lon})["name"];
    );
    out center;
    """
    url = "https://overpass-api.de/api/interpreter"
    try:
        response = requests.post(url, data={'data': query}, timeout=60)
        if response.ok:
            return response.json().get("elements", [])
    except Exception as e:
        print(f"Overpass error at ({lat}, {lon}): {e}")
    return []

def try_enrich_car_wash(row, radius=50):
    if not isinstance(row.get("name", ""), str) or not row["name"].startswith("Unnamed"):
        return row

    lat, lon = row["lat"], row["lon"]
    print(f"🔍 Enriching unnamed at ({lat:.5f}, {lon:.5f})")
    time.sleep(1.5)

    # Nearby search
    elements = get_nearby_named_elements(lat, lon, radius)
    for el in elements:
        tags = el.get("tags", {})
        name_candidate = tags.get("name", "")
        if "car" in name_candidate.lower() or "wash" in name_candidate.lower():
            row["name"] = name_candidate
            row["website"] = tags.get("website", row.get("website"))
            row["contact:phone"] = tags.get("contact:phone", row.get("contact:phone"))
            row["contact:email"] = tags.get("contact:email", row.get("contact:email"))
            row["contact:facebook"] = tags.get("contact:facebook", row.get("contact:facebook"))
            break

    # Add plus code
    try:
        row["pluscode"] = olc.encode(lat, lon)
    except Exception:
        row["pluscode"] = None

    # Add full address
    address = reverse_geocode(lat, lon)
    if address:
        row["address"] = address

    return row
